### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M12D01', 'UTC 16:30')


# Settings

In [2]:
MAXTASKS = 1000
CANCELTASKS = 1

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,error_message,id,output_url,progress,queuetime(min),runtime(min),source_url,start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,NaN,1512144773617,PIrrWN_yearY2005M12V01,NaN,QOEXS22F5LKXXXXWD3HSV44F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145611045
1,NaN,1512144771953,PIrrWN_yearY2004M12V01,NaN,GAJU6GIKJLAOITZS6T7CNIHZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145612965
2,NaN,1512144770219,PIrrWN_yearY2003M12V01,NaN,OMSEWUMO3ESBHR34CZXXDYST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145614638
3,NaN,1512144768519,PIrrWN_yearY2002M12V01,NaN,EY3AR47WPB5BD2VMHWO7LAZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145616272
4,NaN,1512144766782,PIrrWN_yearY2001M12V01,NaN,POWUOHSHAAHLQGDUY6EV3N3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145618206
5,NaN,1512144765102,PIrrWN_yearY2000M12V01,NaN,WU5WZCEWOD3ZJCHW4ESYBJ5U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145619727
6,NaN,1512144763510,PIrrWN_yearY1999M12V01,NaN,HXI5JCTOPI5S4FPBXPWBZTYC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145621529
7,NaN,1512144761760,PIrrWN_yearY1998M12V01,NaN,MHRUFKEKDPTTWUEMK5XQKKWP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145623254
8,NaN,1512144760063,PIrrWN_yearY1997M12V01,NaN,F5WPVA2UIC5AQCGP4JTT2VHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145624988
9,NaN,1512144758309,PIrrWN_yearY1996M12V01,NaN,IVGVXHK5SDBB4VI35OVXJAT3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1512145626930


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)

<Task EXPORT_IMAGE: PIndWN_yearY2002M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY2002M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY2001M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY2001M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY2000M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY2000M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY1999M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY1999M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY1998M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY1998M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY1997M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY1997M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY1996M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY1996M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY1995M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN_yearY1995M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWN_yearY1994M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWN

<Task EXPORT_IMAGE: PIndWW_yearY1985M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1985M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1984M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1984M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1983M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1983M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1982M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1982M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1981M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1981M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1980M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1980M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1979M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1979M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1978M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW_yearY1978M12V01 (READY)>
<Task EXPORT_IMAGE: PIndWW_yearY1977M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PIndWW

<Task EXPORT_IMAGE: PDomWN_yearY1968M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1968M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1967M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1967M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1966M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1966M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1965M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1965M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1964M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1964M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1963M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1963M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1962M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1962M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1961M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN_yearY1961M12V01 (READY)>
<Task EXPORT_IMAGE: PDomWN_yearY1960M12V01 (READY)>
canceling <Task EXPORT_IMAGE: PDomWN